In [1]:
import BioSimSpace as bss
import sys
import os

/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)



Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



In [29]:
def run_process(system, md_protocol):
    """
    @param system: solvated system (BSS object)
    @param md_protocol: BSS protocolBSS protocol
    @return the processed system.
    """
    process = bss.Process.Gromacs(system, md_protocol)
    process.setArg("-nt", 1)
    print(process.getArgs())
    print(process.getOutput)
    process.start()
    process.wait()
    if process.isError():
        print(process.stdout())
        print(process.stderr())
        raise _Exceptions.ThirdPartyError("The process exited with an error!")
    system = process.getSystem()
    return system


In [34]:
minimisation_steps = 500
runtime_short_nvt = 50  # ps
runtime_nvt = 50  # ps
runtime_npt = 200  # ps

# print(f"program: {sys.argv[0]}, index: {sys.argv[1]}")
# index = int(sys.argv[1])
index = 0
ligand_stream = open("execution_model/ligands.dat", "r")
ligand_lines = ligand_stream.readlines()
ligand_name = ligand_lines[index].rstrip()

ligand_solvated = bss.IO.readMolecules([f"execution_model/inputs/ligands/{ligand_name}_ligand_solvated.prm7",
                                        f"execution_model/inputs/ligands/{ligand_name}_ligand_solvated.rst7"])
system_solvated = bss.IO.readMolecules([f"execution_model/inputs/ligands/{ligand_name}_system_solvated.prm7",
                                        f"execution_model/inputs/ligands/{ligand_name}_system_solvated.rst7"])
print("---------------------------")
print("Working on solvated ligand.")
print(f"Minimising in {minimisation_steps} steps..")
protocol = bss.Protocol.Minimisation(steps=minimisation_steps)
minimised = run_process(ligand_solvated, protocol)
print("Finished minimisation.")

print(f"NVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms.")
protocol = bss.Protocol.Equilibration(
                                    runtime=runtime_short_nvt*bss.Units.Time.picosecond,
                                    temperature_start=0*bss.Units.Temperature.kelvin,
                                    temperature_end=300*bss.Units.Temperature.kelvin,
                                    restraint="all"
                                )
restrained_nvt = run_process(minimised, protocol)
print("worked")

---------------------------
Working on solvated ligand.
Minimising in 10 steps..
OrderedDict([('mdrun', True), ('-v', True), ('-deffnm', 'gromacs'), ('-nt', 1)])
<bound method Process.getOutput of BioSimSpace.Process.Gromacs(<BioSimSpace.System: nMolecules=666>, BioSimSpace.Protocol.Minimisation(steps=10), exe='/opt/bin/bin/gmx', name='gromacs', work_dir='/tmp/tmp1jrs4596', seed=None)>
Finished minimisation.
NVT equilibration for 5 ps while restraining all non-solvent atoms.
OrderedDict([('mdrun', True), ('-v', True), ('-deffnm', 'gromacs'), ('-nt', 1)])
<bound method Process.getOutput of BioSimSpace.Process.Gromacs(<BioSimSpace.System: nMolecules=666>, BioSimSpace.Protocol.Equilibration(timestep=2.0000 fs, runtime=5.0000 ps, temperature_start=0.0100 K, temperature_end=300.0000 K, pressure=None, report_interval=100, restart_interval=500, restraint='all', force_constant=10.00), exe='/opt/bin/bin/gmx', name='gromacs', work_dir='/tmp/tmpre11y_fe', seed=None)>
worked


more: stat of logfile failed: No such file or directory


In [26]:
#!ls -l /tmp/tmpwfej5b29

total 232
-rw-rw-r-- 1 jguven jguven     53 May 27 17:00 gromacs.err
-rw-rw-r-- 1 jguven jguven     13 May 27 17:00 gromacs.err.offset
-rw-rw-r-- 1 jguven jguven 108956 May 27 17:00 gromacs.gro
-rw-rw-r-- 1 jguven jguven  13914 May 27 17:00 gromacs.log
-rw-rw-r-- 1 jguven jguven    186 May 27 17:00 gromacs.mdp
-rw-rw-r-- 1 jguven jguven   2828 May 27 17:00 gromacs.out
-rw-rw-r-- 1 jguven jguven  10404 May 27 17:00 gromacs.out.mdp
-rw-rw-r-- 1 jguven jguven     15 May 27 17:00 gromacs.out.offset
-rw-rw-r-- 1 jguven jguven  11653 May 27 17:00 gromacs.top
-rw-rw-r-- 1 jguven jguven  61200 May 27 17:00 gromacs.tpr
-rw-rw-r-- 1 jguven jguven     95 May 27 17:00 README.txt


In [27]:
!cat /tmp/tmpwfej5b29/gromacs.log

                      :-) GROMACS - gmx mdrun, 2021.3 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur